In [80]:
import argparse
import pandas as pd


BASE_DIR = "/home/timot/Classes/ece511/ECE511-Dynamic-Cache-Decompression/results/" # change to your dir



In [81]:
def convert_str_to_num(s):
    """Convert string to either float or int"""
    try:
        return int(s)
    except ValueError:
        return float(s)



    

    

In [82]:
# single core
metrics_wanted = ["overallMissRate::total", "bwTotal::total",
                  "ipc"]


In [83]:

def get_metrics(lines):
    metrics = {}
    for line in lines:
        line_lowercase = line.lower()
        
  
        if "miss" in line_lowercase or "hit" in line_lowercase or "system.l3" in line_lowercase or "ipc" in line_lowercase:
            # the line is interesting to us. extract the first and second non-whitespace words
            words = line.split()
            metric = words[0]
            value = words[1]
            if metric not in metrics:
                metrics[metric] = convert_str_to_num(words[1])

    rows = []
    for k,v in metrics.items():
        rows.append([k,v])

    metrics_df = pd.DataFrame(rows, columns=["metric", "value"])
    return metrics_df


In [84]:


def over_threshold(percent, threshold):
    # return True if the percent difference between a and b is greater than the threshold
    # ensure that the first number is greater than the second
    return abs(percent) > threshold



In [85]:

def get_percent_diff(a, b):
    # return the percent difference between a and b
    # ensure that the first number is greater than the second
    if a < b:
        a, b = b, a
    try:
        percent_diff = (a - b) / a
        return percent_diff
    except ZeroDivisionError:
        return 0


In [86]:


def get_file_lines(file):
    with open(file, "r") as f:
        lines = f.readlines()
    return lines

    


In [87]:

def get_same_metrics_singlecore(df):
    # metrics are in the order 
    # system.cpu.<metric we are interested in>
    # we want to get the metrics that are the same for all cores
    # so we can compare them
    # we will assume that the first core is the same as the rest
    

    

    # convert to a dict
    rows=[]
    for idx, row in df.iterrows():
        value = row.value
        if row.metric.startswith("system.cpu"):
            metric = row.metric.replace("system.cpu.", "")
            rows.append([metric, value])
        elif row.metric.startswith("system.l3"):
            metric = row.metric.replace("system.l3", "l3")
            rows.append([metric, value])

    # make a dataframe
    first_core = pd.DataFrame(rows, columns=["metric", "value"])
    return first_core




In [88]:
def get_same_metrics_multicore(df, num_cores, avg=False):
    # metrics are in the order 
    # system.cpu[num for cpu core].<metric we are interested in>
    # we want to get the metrics that are the same for all cores
    # so we can compare them
    # we will assume that the first core is the same as the rest
    

    cores = [f"system.cpu{i}." for i in range(num_cores)]
    # get the first core
    core_metric_rows = {}
    for core in cores:
        for idx, row in df.iterrows():
            value = row.value
            if row.metric.startswith(core):
                metric = row.metric.replace(core, "")
                if metric not in core_metric_rows:
                    core_metric_rows[metric] = value
                else:
  
                    core_metric_rows[metric] += value

    rows = []
    for k,v in core_metric_rows.items():
        if avg:
            core_metric_rows[k] = v / num_cores
        rows.append([k, core_metric_rows[k]])

    
    # L3
    for idx, row in df.iterrows():
        value = row.value
        if row.metric.startswith("system.l3"):
            metric = row.metric.replace("system.l3", "l3")
            rows.append([metric, value])

    # make a dataframe
    cores_df = pd.DataFrame(rows, columns=["metric", "value"])
    return cores_df
        

                
get_same_metrics_multicore(file4_metrics, 4, avg=True)


,metric,value
0,numIssuedDist::samples,7.009843e+06
1,numIssuedDist::mean,2.019188e+00
2,numIssuedDist::stdev,2.101963e+00
3,numIssuedDist::underflows,0.000000e+00
4,numIssuedDist::0,3.390232e+06
...,...,...
1476,l3.tags.dataAccesses,3.965413e+06
1477,l3.tags.evictionsReplacement::0,5.144500e+04
1478,l3.tags.evictionsReplacement::1,0.000000e+00
1479,l3.tags.evictionsReplacement::2,0.000000e+00


In [89]:

# file1 = parser.add_argument("--file1", help="File 1", required=True)
# file4 = parser.add_argument("--file4", help="File 4", required=True)

file1 = BASE_DIR + "riscv/FFT/1/Base16Delta8/stats.txt"
file4 = BASE_DIR + "riscv/FFT/4/Base16Delta8/stats.txt"

file1_lines = get_file_lines(file1)
file4_lines = get_file_lines(file4)

file1_metrics = get_metrics(file1_lines)
file4_metrics = get_metrics(file4_lines)

# file1_metrics

cpu1core = get_same_metrics_singlecore(file1_metrics)
cpu1core
# get the data frame from file1 and file4


,metric,value
0,numIssuedDist::samples,2.740277e+07
1,numIssuedDist::mean,1.623901e+00
2,numIssuedDist::stdev,2.065006e+00
3,numIssuedDist::underflows,0.000000e+00
4,numIssuedDist::0,1.312692e+07
...,...,...
576,l3.tags.dataAccesses,9.463441e+06
577,l3.tags.evictionsReplacement::0,5.040100e+04
578,l3.tags.evictionsReplacement::1,0.000000e+00
579,l3.tags.evictionsReplacement::2,0.000000e+00


In [90]:
cpu4core = get_same_metrics_multicore(file4_metrics, 4, avg=True)
cpu4core



,metric,value
0,numIssuedDist::samples,7.009843e+06
1,numIssuedDist::mean,2.019188e+00
2,numIssuedDist::stdev,2.101963e+00
3,numIssuedDist::underflows,0.000000e+00
4,numIssuedDist::0,3.390232e+06
...,...,...
1475,l3.tags.dataAccesses,3.965413e+06
1476,l3.tags.evictionsReplacement::0,5.144500e+04
1477,l3.tags.evictionsReplacement::1,0.000000e+00
1478,l3.tags.evictionsReplacement::2,0.000000e+00


In [91]:
def find_metric(metric, df):
    # find the metric in the dataframe
    # return the value
    for idx, row in df.iterrows():
        if metric in row.metric:
            return row.value
    return None

In [92]:
# single core
metrics_wanted = ["overallMissRate::total", "ipc"]

In [93]:
for metric in metrics_wanted:
    print(find_metric(metric, cpu4core))

0.07793725
1.9310097500000003
